# Inference time for customs

In [69]:
import torch

In [70]:
from sklearn.preprocessing import OneHotEncoder

In [71]:
from utils_transformer import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
link2="predictions_2023-07-29T02.csv"
link1="results_analysis_v1.csv"
from sentence_transformers import SentenceTransformer, util
import torch
device= 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
from tqdm import tqdm

cuda


In [72]:
model_customs= Encoder_Smash(
vocab_size= 28996,
    max_len= 512,
    d_k= 16,
    d_model=64,
    n_heads=4,
    n_layers=2,
    n_classes=351,
    dropout_prob=0.2
)
model_customs.to(device)

Encoder_Smash(
  (embedding): Embedding(28996, 64)
  (pos_encoding): PositionalEncodingsyeah(
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (all_transformer_blocks): Sequential(
    (0): TransformadorBlock(
      (layern1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (layern2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (mha_created): MultiheadAttention(
        (keys): Linear(in_features=64, out_features=64, bias=True)
        (value): Linear(in_features=64, out_features=64, bias=True)
        (query): Linear(in_features=64, out_features=64, bias=True)
        (fin_layer): Linear(in_features=64, out_features=64, bias=True)
      )
      (ann): Sequential(
        (0): Linear(in_features=64, out_features=256, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=256, out_features=64, bias=True)
        (3): Dropout(p=0.2, inplace=False)
      )
      (dropoutlay): Dropout(p=0.2, inplace=False)
    )
    (1): TransformadorBl

In [73]:
model_customs.load_state_dict(torch.load('best_model.pt'))

<All keys matched successfully>

In [190]:
res_df = pd.read_csv(link1)
act_df = pd.read_csv(link2)
model_feed=res_df[['medium_description_pim','HS_Code_x','Val-NoVal','ProductNum']]
model_feed_val= model_feed[model_feed['Val-NoVal']=='validated']
df_for_onehot =model_feed_val['HS_Code_x'].value_counts().reset_index()
df_for_onehot['index_3']=df_for_onehot.index
model_feed_val2=model_feed_val.merge(df_for_onehot, left_on= 'HS_Code_x', right_on='index', how='inner')
y= model_feed_val2['index_3']
onhe=OneHotEncoder(handle_unknown='ignore', sparse=False).fit(y.to_numpy().reshape(-1,1))

C:\Users\SESA608167.NAM\AppData\Local\Temp\ipykernel_40932\4218247528.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  res_df = pd.read_csv(link1)


In [93]:
from datasets import load_dataset
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, DataCollatorWithPadding
from torch.utils.data import DataLoader

In [193]:
raw_ds=DatasetDict({'train':
    Dataset.from_pandas(model_feed_val2)
           
} )
checkpoint= 'distilbert-base-cased'
tokenizer= AutoTokenizer.from_pretrained(checkpoint)
def tokenize_func(batch):
    return tokenizer(batch['medium_description_pim'], truncation=True)

In [194]:
token_ds= raw_ds.map(tokenize_func, batched=True)
data_collator= DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/105093 [00:00<?, ? examples/s]

In [195]:
data_collator

DataCollatorWithPadding(tokenizer=DistilBertTokenizerFast(name_or_path='distilbert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True), padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

In [196]:
token_ds_backup=token_ds
token_ds=token_ds.remove_columns(['medium_description_pim','Val-NoVal','__index_level_0__', 'HS_Code_x_x','HS_Code_x_y', 'index','ProductNum'])
token_ds=token_ds.rename_column('index_3','labels')

In [197]:
train_loader= DataLoader(
token_ds['train'],
    shuffle=False,
    batch_size=32,
    collate_fn=data_collator
)

In [198]:
# Check chak after padded
for batch in train_loader:
    print(batch['input_ids'][19])
    print(len(batch['input_ids'][19]))
    break

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([  101, 12008,  8223, 14895, 21484,   131, 12786,  5822,   117,  5096,
         3962,   131,  4372,  1568,  6262,   117,  6475,   131,  1398,   117,
          123,   118, 24781,   117,   121,   118,  1620,   140,   117,   138,
        19515,   131,   121,   119,   125,   110,   117, 23676,  2137, 12882,
         1568,   118,  2539,   121,   120,  1620,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0])
56


In [199]:
for batch in train_loader:
    print(len(batch['input_ids'][0]))
    break

56


In [200]:
token_ds_backup['train']['medium_description_pim'][19]

'Temp Sensor: Duct, Probe: 6000mm, Systems: All, 2-Wire, 0-100 C, Acc: 0.4 %, STD400-60 0/100'

In [201]:
print(token_ds_backup['train']['index_3'][19])
print(token_ds_backup['train']['index'][19])

58
902519


In [202]:
df_for_onehot[df_for_onehot['index']=='902519']

,index,HS_Code_x,index_3
58,902519,130,58


In [203]:
n_correct= 0.0
n_correct_ver2= 0.0
n_total=0.0
output_list=[]
output_list_by_batch=[]
for batch in train_loader:
    
    batch = {k: v.to(device) for k, v in batch.items()}
    the_number=batch['labels']
    the_number2=np.array(the_number.cpu()).reshape(-1,1)
    torched_labels= torch.tensor(onhe.transform(the_number2)).to(device)
    outputs= model_customs(batch['input_ids'], batch['attention_mask'])
    to_list=np.array(outputs.detach().cpu())
    output_list_by_batch.append(to_list)
    for single_out in to_list:
        output_list.append(single_out)
    _, predictions = torch.max(outputs,1)
    n_correct += (predictions==batch['labels']).sum().item()
    array3=np.array(torch.argmax(outputs, axis=1).detach().cpu())
    array4=np.array(torch.argmax(torched_labels, axis=1).detach().cpu())
    n_correct_ver2+= sum(array3==array4)
    n_total += batch['labels'].shape[0]
test_acc= n_correct/n_total
test_acc2=n_correct_ver2/n_total
print(f' Text acc: {test_acc}')

 Text acc: 0.9372460582531662


In [204]:
n_correct_ver2

98498.0

In [205]:
n_correct

98498.0

In [206]:
n_total

105093.0

In [209]:
output_list[19]

array([ -8.521733 ,  -5.061152 ,  -9.768784 ,  -8.632462 , -12.992258 ,
       -13.704337 , -11.49689  ,  -8.324841 ,  -6.7064657, -11.221553 ,
       -11.345498 ,  -7.6209126, -18.870478 ,  -9.660956 ,  -9.825319 ,
        -7.041059 , -13.603882 , -15.76269  , -11.703709 , -10.920628 ,
       -21.546364 ,  -7.4851174, -19.613148 ,  -4.8723583,  -9.712845 ,
        -9.389515 ,  -8.176589 , -10.261168 , -13.251736 , -17.693262 ,
       -11.296856 , -11.517308 ,  -9.124899 , -15.991551 ,  -8.33569  ,
       -15.915739 , -17.054216 , -13.400009 , -15.377597 , -16.817474 ,
       -23.030136 ,  -3.7993703, -14.037158 ,  -8.579782 ,  -6.3065486,
       -20.691301 , -11.962691 ,  -7.609716 ,  -9.901137 , -11.840009 ,
       -10.012627 , -15.871306 ,  -9.035135 , -17.870354 ,  -9.967959 ,
       -12.2641   , -17.922886 , -13.21751  ,   4.466721 , -12.361855 ,
       -19.161203 , -13.816442 , -16.33972  , -21.177174 ,  -6.1720934,
       -11.402507 ,  -8.800237 , -11.074249 , -13.862703 , -15.1

In [210]:
output_list[9].argmax()

58

In [211]:
argmax_list=[]
for i in output_list:
    argmax_list.append(i.argmax())

In [212]:
len(argmax_list)

105093

In [213]:
model_feed_val2['prediction']=argmax_list

In [214]:
model_feed_val2.iloc[19]

medium_description_pim    Temp Sensor: Duct, Probe: 6000mm, Systems: All...
HS_Code_x_x                                                          902519
Val-NoVal                                                         validated
ProductNum                                                        006920761
index                                                                902519
HS_Code_x_y                                                             130
index_3                                                                  58
prediction                                                               58
Name: 19, dtype: object

In [215]:
df_for_onehot[df_for_onehot['index']=='902519']

,index,HS_Code_x,index_3
58,902519,130,58


In [216]:
df_for_onehot.columns= ['oh_index','oh_count','oh_code']

In [217]:
model_feed_val3=model_feed_val2.merge(df_for_onehot, left_on= 'prediction', right_on='oh_code', how='inner')

In [218]:
model_feed_val3.iloc[19]

medium_description_pim    Temp Sensor: Duct, Probe: 400mm, Systems: All,...
HS_Code_x_x                                                          902519
Val-NoVal                                                         validated
ProductNum                                                        006920861
index                                                                902519
HS_Code_x_y                                                             130
index_3                                                                  58
prediction                                                               58
oh_index                                                             902519
oh_count                                                                130
oh_code                                                                  58
Name: 19, dtype: object

In [219]:
model_feed_val3['MATCH']=model_feed_val3['oh_index']==model_feed_val3['HS_Code_x_x']

In [220]:
sum(model_feed_val3['MATCH'])

98498

In [221]:
sum(model_feed_val3['MATCH'])/len(model_feed_val3['MATCH'])

0.9372460582531662

In [223]:
model_feed_val3.columns

Index(['medium_description_pim', 'HS_Code_x_x', 'Val-NoVal', 'ProductNum',
       'index', 'HS_Code_x_y', 'index_3', 'prediction', 'oh_index', 'oh_count',
       'oh_code', 'MATCH'],
      dtype='object')

In [224]:
model_feed_val3.columns= ['Medium_desc_model','HS_Code_model','Val-NoVal_model','ProductNum',
                          'index_model','HS_CodeY','index_3','prediction','oh_index','oh_count','oh_code','MATCH']

In [226]:
title= 'transformer_encoder_resultsv1.csv'
model_feed_val3.to_csv(title)

In [165]:
res_df.columns

Index(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'ProductNum',
       'GlobalHsNum', 'CountryHsNum', 'BusinessUnit', 'AssuranceLevel',
       'CountryCode', 'Datasource', 'ECCNumber', 'HSNum2',
       'commercial_reference_pim', 'medium_description_pim', 'gdp_code_pim',
       'gdp_code_pmo', 'short_description_pmo', 'prediction',
       'prediction_probability', 'HS_Code_x', 'Val-NoVal', 'Medium_desc_Trunc',
       'desc_len', 'HS_Code_y', 'Count_HSCode', 'Match', 'pred_label'],
      dtype='object')

In [159]:
res_df[res_df['Val-NoVal']=='validated'].shape

(105093, 27)

In [160]:
len(model_feed_val3['MATCH'])

105093

In [164]:
model_feed_val3.columns= ['Medium_desc_model','HS_Code_model','Val-NoVal_model','index_model','HS_CodeY','index_3','prediction','oh_index','oh_count','oh_code','MATCH']

In [166]:
model_feed_val4=model_feed_val3.merge(res_df, left_on= 'Medium_desc_model', right_on='medium_description_pim', how='inner')

In [167]:
title= 'transformer_encoder_resultsv1.csv'
model_feed_val4.to_csv(title)

In [178]:
result_pm_fin=model_feed_val4.groupby('ProductNum')['HS_Code_model'].count()

In [183]:
result_pm_fin.sort_values()

ProductNum
0011055002600        1
MINBAC000002050      1
MINBAC000002049      1
MINBAC000002048      1
MINBAC000002047      1
                  ... 
PFXGP4601TMDF3     189
PFXGP4601TMDF2C    189
PFXGP4601TMDF2     189
PFXGP4603TADF3C    189
PFXGP4601TMAF3C    189
Name: HS_Code_model, Length: 111315, dtype: int64

In [184]:
res_df[res_df['ProductNum']=='PFXGP4601TMAF3C']

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ProductNum,GlobalHsNum,CountryHsNum,BusinessUnit,AssuranceLevel,CountryCode,Datasource,...,prediction,prediction_probability,HS_Code_x,Val-NoVal,Medium_desc_Trunc,desc_len,HS_Code_y,Count_HSCode,Match,pred_label
125398,125398,124161,170261,PFXGP4601TMAF3C,853710,8537109199,SE,3,NL,MDG,...,853710,1.0,853710,validated,Graphic Display Panel,21,853710,123.0,True,>95%


In [185]:
res_df[res_df['medium_description_pim']=='Graphic Display Panel']

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ProductNum,GlobalHsNum,CountryHsNum,BusinessUnit,AssuranceLevel,CountryCode,Datasource,...,prediction,prediction_probability,HS_Code_x,Val-NoVal,Medium_desc_Trunc,desc_len,HS_Code_y,Count_HSCode,Match,pred_label
125341,125341,124104,170201,PFXGP3300HLAD,852852,8528529100,SE,2,NL,MDG,...,853710,0.999999,852852,Non validated,Graphic Display Panel,21,853710,123.0,False,>95%
125342,125342,124105,170202,PFXGP3300HLADGK,853710,8471410000,SE,1,NL,MDG,...,853710,0.999999,847141,Non validated,Graphic Display Panel,21,853710,123.0,False,>95%
125343,125343,124106,170203,PFXGP3300HLADRK,853710,8471410000,SE,1,NL,MDG,...,853710,0.999999,847141,Non validated,Graphic Display Panel,21,853710,123.0,False,>95%
125344,125344,124107,170204,PFXGP3300HSADRK,852852,8528529100,SE,2,NL,MDG,...,853710,0.999999,852852,Non validated,Graphic Display Panel,21,853710,123.0,False,>95%
125345,125345,124108,170205,PFXGP3310HTAD,852852,8528529100,SE,2,NL,MDG,...,853710,0.999999,852852,Non validated,Graphic Display Panel,21,853710,123.0,False,>95%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227611,227611,224580,288580,PFXSP5700TPDF2C,852859,8529909200,SE,5,NL,MDG,...,853710,0.999827,852990,validated,Graphic Display Panel,21,853710,123.0,False,>95%
227612,227612,224581,288581,PFXSP5700TPDF6,852859,8529909200,SE,5,NL,MDG,...,853710,0.999827,852990,validated,Graphic Display Panel,21,853710,123.0,False,>95%
227613,227613,224582,288582,PFXSP5700TPDF7C,852859,8529909200,SE,5,NL,MDG,...,853710,0.999827,852990,validated,Graphic Display Panel,21,853710,123.0,False,>95%
227614,227614,224583,288583,PFXSP5700TPDF8,852859,8529909200,SE,5,NL,MDG,...,853710,0.999827,852990,validated,Graphic Display Panel,21,853710,123.0,False,>95%


In [187]:
model_feed_val3[model_feed_val3['Medium_desc_model']=='Graphic Display Panel']

,Medium_desc_model,HS_Code_model,Val-NoVal_model,index_model,HS_CodeY,index_3,prediction,oh_index,oh_count,oh_code,MATCH
43783,Graphic Display Panel,853710,validated,853710,3529,8,8,853710,3529,8,True
43784,Graphic Display Panel,853710,validated,853710,3529,8,8,853710,3529,8,True
43785,Graphic Display Panel,853710,validated,853710,3529,8,8,853710,3529,8,True
43786,Graphic Display Panel,853710,validated,853710,3529,8,8,853710,3529,8,True
43787,Graphic Display Panel,853710,validated,853710,3529,8,8,853710,3529,8,True
...,...,...,...,...,...,...,...,...,...,...,...
46134,Graphic Display Panel,852990,validated,852990,109,65,8,853710,3529,8,False
46135,Graphic Display Panel,852990,validated,852990,109,65,8,853710,3529,8,False
46136,Graphic Display Panel,852990,validated,852990,109,65,8,853710,3529,8,False
46137,Graphic Display Panel,852990,validated,852990,109,65,8,853710,3529,8,False


In [191]:
model_feed_val2[model_feed_val2['medium_description_pim']=='Graphic Display Panel']

,medium_description_pim,HS_Code_x_x,Val-NoVal,ProductNum,index,HS_Code_x_y,index_3
84412,Graphic Display Panel,853710,validated,PFXGP4201TADF3C,853710,3529,8
84413,Graphic Display Panel,853710,validated,PFXGP4201TADWF2,853710,3529,8
84414,Graphic Display Panel,853710,validated,PFXGP4201TADWF3C,853710,3529,8
84415,Graphic Display Panel,853710,validated,PFXGP4203TADF3C,853710,3529,8
84416,Graphic Display Panel,853710,validated,PFXGP4301TADCF2,853710,3529,8
...,...,...,...,...,...,...,...
102820,Graphic Display Panel,852990,validated,PFXSP5660TPDF7,852990,109,65
102821,Graphic Display Panel,852990,validated,PFXSP5700TPDF2C,852990,109,65
102822,Graphic Display Panel,852990,validated,PFXSP5700TPDF6,852990,109,65
102823,Graphic Display Panel,852990,validated,PFXSP5700TPDF7C,852990,109,65


In [192]:
1

1

In [228]:
model_feed_val2[model_feed_val2['medium_description_pim']== 'CANALIS KT 5X3200AL FEEDER LENGTH']

,medium_description_pim,HS_Code_x_x,Val-NoVal,ProductNum,index,HS_Code_x_y,index_3,prediction
39413,CANALIS KT 5X3200AL FEEDER LENGTH,854449,validated,KTA3200ET53E,854449,2216,12,12
39414,CANALIS KT 5X3200AL FEEDER LENGTH,854449,validated,KTA3200ET61A,854449,2216,12,12
39415,CANALIS KT 5X3200AL FEEDER LENGTH,854449,validated,KTA3200ET62C,854449,2216,12,12
39420,CANALIS KT 5X3200AL FEEDER LENGTH,854449,validated,KTA3200ET73E,854449,2216,12,12
40115,CANALIS KT 5X3200AL FEEDER LENGTH,854449,validated,KTA3200ET62B,854449,2216,12,12
40120,CANALIS KT 5X3200AL FEEDER LENGTH,854449,validated,KTA3200ET73F,854449,2216,12,12
40870,CANALIS KT 5X3200AL FEEDER LENGTH,854449,validated,KTA3200ET53F,854449,2216,12,12
40871,CANALIS KT 5X3200AL FEEDER LENGTH,854449,validated,KTA3200ET63D,854449,2216,12,12


In [230]:
df_for_onehot[df_for_onehot['oh_code']==12]

,oh_index,oh_count,oh_code
12,854449,2216,12
